# Leitor de RSS - CEPEA

Esse _script_ faz a leitura do RSS para ver as últimas publicações do CEPEA.

O RSS possui uma limitação de meses, portanto, só as últimas notícias estarão nela.

## 1. Funcionamento

Por ser feito em [Beautiful Soup](https://pypi.org/project/beautifulsoup4/) (`bs4`) é computacionalmente mais leve e fácil de trabalhar.

## 2. Requisitos

Será utilizado o [requests](https://pypi.org/project/requests/) para o acesso [HTPP](https://pt.wikipedia.org/wiki/Hypertext_Transfer_Protocol) e o Beautiful Soup para ler as informações contidas no site.

### 2.1. Bibliotecas

* `requests`: Acesso a sites por HTTP;
* Beautiful Soup (`bsf4`): Interpretação do site;
* `pandas`: Criação e manuseio de dataframes, e;
* `os`: Navegação entre as pastas do sistema operacional.

In [ ]:
# Importação das bibliotecas (packages)
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import time

## 3. Inicialização das Variáveis

O URL base do RSS do CEPEA:

* http://www.cepea.esalq.usp.br/rss.php

In [ ]:
# URL do feed RSS
rss_url = "http://www.cepea.esalq.usp.br/rss.php"

## 4. Configuração do `requests`

In [ ]:
# Definir um cabeçalho User-Agent
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
}

## 5. Leitura do Feed

In [ ]:
# Fazer a requisição com o cabeçalho modificado
response = requests.get(rss_url, headers=headers)

# Verificar se a requisição foi bem sucedida
if response.status_code == 200:
    # Parsear o conteúdo XML do RSS com o BeautifulSoup
    soup = BeautifulSoup(response.content, 'xml')
    # Extrair os itens do feed
    items = soup.find_all('item')

    # Listas para armazenar os dados
    dates = []
    products = []
    titles = []
    links = []

    # Filtrar e coletar os dados que começam com "AÇÚCAR/CEPEA" ou "ETANOL/CEPEA"
    for item in items:
        title = item.find('title').text
        link = item.find('link').text
        pub_date = item.find('pubDate').text
        
        if title.startswith("AÇÚCAR/CEPEA"):
            product = "AÇÚCAR/CEPEA"
            title = title.replace("AÇÚCAR/CEPEA: ", "")
        elif title.startswith("ETANOL/CEPEA"):
            product = "ETANOL/CEPEA"
            title = title.replace("ETANOL/CEPEA: ", "")
        else:
            continue
        
        dates.append(pub_date)
        products.append(product)
        titles.append(title)
        links.append(link)

    # Criar o DataFrame
    df = pd.DataFrame({
        'Data': dates,
        'Produto': products,
        'Título': titles,
        'URL': links
    })

else:
    print(f"Erro: {response.status_code} - Não foi possível acessar o feed RSS.")

## 6. Visualização Tabular do RSS

As primeiras 5 linhas da tabela.

In [ ]:
# Primeiras 5 linhas da tabela
df.head(5)

## 7. Exportação

In [ ]:
# Caminho completo para o arquivo de exportação
file_path = os.path.join("01-data", time.strftime('%Y-%m-%d') + ' ' + 'cepea_rss.csv')

# Exporta na pasta 01-data
df.to_csv(file_path, index=False, encoding='utf-8-sig')